# Introduction to stochastic optimization

The content in this class is based on the book "Uma Introducção à Otimização sob Incerteza" by Humberto Bortolossi and Bernardo Pagnoncelli.

## About stochastic optimization
    
Most real-life problems bring uncertainties in themselves: they are inherent in virtually all systems related to actuarial science, economics, meteorology, demography, ecology, and so on. Nowadays, problems involving interactions between man, nature and technology are subject to rapid change, which increases uncertainty. Each new technological revolution brings new challenges to the knowledge established up to then. Even in the deterministic context, there are systems that are complex, which do not allow precise measurement of their parameters.
   
The area of stochastic optimization (also known as optimization under uncertainty) studies methods to address such situations: they incorporate uncertainties in modeling through the inclusion of random variables of known probability distributions. The aim is to find solutions that are acceptable for all possible realizations of the modeled random variables.
   
## The farmer problem

John is a farmer with $500$ Hectares (ha) of land available for cultivation. Remember that $500 ha$ equals $5000000 m^2$. He is a specialist in three crops: wheat, maize, and sugarcane. During the winter, he has to decide how much land will be dedicated to each of the three cultures. The figure below shows two possible land divisions.

![Figure 1](Figure1.png)

In addition to the size of its property, John has other constraints to consider. He is also the owner of livestock, which needs to be fed. Their livestock needs at least 200 tons (T) of wheat and 240 tons of corn for the ration.  Aside from the wheat and corn produced on his land, he can buy these products from other producers in the local market. Their excess production can be sold to wholesalers, for the price and much smaller because of the profit margin of these traders.

Sugar cane is a crop exclusively for profit: all its production is sold to wholesalers at 36 dollars per tonne (dollars/T). However, the government imposes a production quota of 6,000 T: any quantity produced above this value should be sold for only 10 dollars/T.

Based on information from previous years, John knows that the average yield of his crops is 2.5, 3.0 and 20 tons per hectare (T/ha). In addition, there is a specific production cost of each crop, which is given in  dollars/ha. The complete model data are shown in the table below:

|                                -  | wheat | corn |      sugarcane     |
|:---------------------------------:|:-----:|:----:|:------------------:|
|            yield (T/ha)           |  2.5  |  3.0 |         20         |
|   production costs (dollars/ha)   |  150  |  230 |         260        |
|     selling price (dollars/T)     |  170  |  150 | 36 ($\leq$ 6000 T) |
|                        -          |   -   |   -  |    10 (> 6000 T)   |
|      buying price (dollars/T)     |  238  |  210 |         -          |
| minimum requirement for livestock |  200  |  240 |         -          |

Remember that the total available land is 500 ha.

To help John decide how to divide his lands in order to maximize his profits, let's formulate a linear optimization problem that describes this situation. Define:

|variable|Meaning                                 |
|--------|:---------------------------------------|
| $x_1$  | hectares dedicated to wheat            |
| $x_2$  | hectares dedicated to corn             |
| $x_3$  | hectares dedicated to sugarcane        |
| $w_1$  | tonnes of wheat sold                   |
| $y_1$  | tonnes of wheat bought                 |
| $w_2$  | tonnes of corn sold                    |
| $y_2$  | tonnes of corn bought                  |
| $w_3$  | tonnes of sugarcane sold at 36 dollars |
| $w_4$  | tonnes of sugarcane sold at 10 dollars |

With this variables we can write the linear programming problem as 

$$
\begin{align}
%
\max  170w_1 +150w_2 +36w_3 +10w_4 -150x_1 - 230 x_2 -&260 x_3 -
238y_1 -210y_2\\
%
\mbox{s.t.: } & \nonumber \\
x_1 + x_2 + x_3 &\leq 500,\\
2.5x_1 + y_1 - w_1 &\geq 200, \\
3x_2 + y_2 - w_2 &\geq 240, \\
w_3 + w_4 &\leq 20 x_3, \\
w_3 &\leq 6000, \\
x_1, x_2, x_3, y_1, y_2, w_1, w_2, w_3, w_4  &\geq 0\\
%
\end{align}
$$

And good news, we can solve this with JuMP. Make sure the `Project.toml` and `Manifest.toml` files are in the same repository as this notebook so we can intall all packages automatically.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Excellent! Everything is good to go!


In [10]:
using JuMP, GLPK

model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[i in 1:3] >= 0)
@variable(model, y[i in 1:2] >= 0)
@variable(model, w[i in 1:4] >= 0)

@constraints(model, begin
    x[1] + x[2] + x[3] <= 500
    2.5*x[1] + y[1] - w[1] >= 200
    3*x[2] + y[2] - w[2] >= 240
    w[3] + w[4] <= 20*x[3]
    w[3] <= 6000
end)

@objective(model, Max, 170*w[1] + 150*w[2] + 36*w[3] + 10*w[4] - 150*x[1] - 230*x[2] - 260*x[3] - 238*y[1] - 210*y[2])
model

A JuMP Model
Maximization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y

In [17]:
optimize!(model)
println("Dedicated area $(JuMP.value.(x))")
println("Total bought $(JuMP.value.(y))")
println("Total sold $(JuMP.value.(w))")
println("Profit: $(JuMP.objective_value(model))")

Dedicated area [120.0, 80.0, 300.0]
Total bought [0.0, 0.0]
Total sold [100.0, 0.0, 6000.0, 0.0]
Profit: 118600.0
